# Twomes data extraction and backup

This JupyterLabs notebook can be used download raw data from a Twomes database (see also [more information how to setup a Twomes server](https://github.com/energietransitie/twomes-backoffice-configuration#jupyterlab)).

Don't forget to install the requirements listed in [requirements.txt](../requirements.txt) first!



## Setting the stage

First several imports and variables need to be defined


### Imports and generic settings

In [ ]:
from datetime import datetime, timedelta
import pytz
import math
import pylab as plt

import pandas as pd
import numpy as np

# usually, two decimals suffice for displaying DataFrames (NB internally, precision may be higher)
pd.options.display.precision = 2

import sys
sys.path.append('../data/')
# sys.path.append('../view/')
# sys.path.append('../analysis/')

%load_ext autoreload
import gc

%matplotlib widget
from plotter import Plot
from filewriter import ExcelWriter as ex

from measurements import Measurements

from tqdm.notebook import tqdm


import logging
logging.basicConfig(level=logging.INFO, 
                    format='%(asctime)s %(levelname)-8s %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    filename='log.txt',
                   )

### Defining which homes, which period 

- which `homes` 
- what the location and timezone is of those homes (currently, we only support one location and timezone for a batch of homes) 
- from which `start_day` to which `end_day` 

In [ ]:
# location: center of Assendorp neighbourhood in Zwolle
lat, lon = 52.50655, 6.09961

# timezone: 
timezone_database = 'UTC'
timezone_homes = 'Europe/Amsterdam'

# Period: maximum period of datacollection
first_day = pytz.timezone(timezone_homes).localize(datetime(2021, 10, 25))
last_day = pytz.timezone(timezone_homes).localize(datetime(2022, 5, 8))

# Period: Shorter period with suitable weather and lots of homes with measurements.
# first_day = pytz.timezone(timezone_homes).localize(datetime(2022, 1, 3))
# last_day = pytz.timezone(timezone_homes).localize(datetime(2022, 1, 31))


# Homes: full set
homes_all = [803422, 805164, 809743, 811308, 815925, 817341, 822479, 829947, 830088, 831062, 839440, 845966, 845997, 846697, 857477, 864296, 873985, 879481, 881611, 886307, 895671, 897349, 899510]

# Homes: subset of homes
homes_3 = [803422, 805164, 809743]

# Homes: single home (virtual home)
homes_single = [886307]

# Homes: single home for gap assessment
# homes = [803422]

# Properties: a single one
twomes_single_property_names = {
    'roomTemp' : 'T_in_avg_degC'
}
twomes_single_property = [k for k in twomes_single_property_names]
twomes_single_property_type = {
    'roomTemp' : 'float32'
}


# Properties: limited set
twomes_limited_properties_types = {
    'eMeterReadingReturnHigh' : 'float64',
    'eMeterReadingReturnLow' : 'float64',
    'eMeterReadingSupplyHigh' : 'float64', 
    'eMeterReadingSupplyLow' : 'float64',
    'eMeterReadingTimestamp' : 'str',
    'gMeterReadingSupply' : 'float64',
    'gMeterReadingTimestamp' : 'str',
    'roomTemp' : 'float32',
    'roomSetpointTemp' : 'float32'
}
twomes_limited_properties = [k for k in twomes_limited_properties_types]

# Properties:  full set
twomes_full_properties_types = {
    'heartbeat' : 'Int16',
    'eMeterReadingReturnHigh' : 'float64',
    'eMeterReadingReturnLow' : 'float64',
    'eMeterReadingSupplyHigh' : 'float64', 
    'eMeterReadingSupplyLow' : 'float64',
    'eMeterReadingTimestamp' : 'str',
    'gMeterReadingSupply' : 'float64',
    'gMeterReadingTimestamp' : 'str',
    'listRSSI' : 'str',
    'boilerTemp1' : 'float32',
    'boilerTemp2' : 'float32',
    'roomTemp' : 'float32',
    'boilerSupplyTemp' : 'float32',
    'isBoilerFlameOn' : 'Int8',
    'isCentralHeatingModeOn' : 'Int8',
    'isDomesticHotWaterModeOn' : 'Int8',
    'maxModulationLevel' : 'Int8',
    'roomSetpointTemp' : 'float32',
    'minModulationLevel' : 'Int8',
    'boilerReturnTemp' : 'float32',
    'relativeModulationLevel' : 'Int8',
    'boilerMaxSupplyTemp' : 'float32',
    'CO2concentration' : 'Int16',
    'relativeHumidity' : 'float32',
    'roomTemp2' : 'float32'}
twomes_full_properties = [k for k in twomes_full_properties_types]

## Getting measurements: 1 property, 1 home

In [ ]:
%%time 
%autoreload 2
df_meas= Measurements.get_raw_measurements(
    homes_single,
    first_day, last_day,
    twomes_single_property,
    timezone_database, timezone_homes
)

In [ ]:
df_meas.info()

In [ ]:
df_meas

## Get measuremens for more properties for 3 homes

In [ ]:
%%time 
df_meas= Measurements.get_raw_measurements(
    homes_3,
    first_day, last_day,
    twomes_limited_properties,
    timezone_database, timezone_homes
)

In [ ]:
df_meas.info()

In [ ]:
df_meas

## Get all measurements for all homes and write to parquet file(s)

### Get measurements for all properties for 23 homes

In [ ]:
%%time 
df_meas = Measurements.get_raw_measurements(
    homes_all,
    first_day, last_day,
    twomes_full_properties,
    timezone_database, timezone_homes
)

In [ ]:
df_meas.info()

In [ ]:
df_meas

### Writing raw measurements to a parquet file

In [ ]:
%%time 
df_meas.to_parquet('twomes_raw_measurements.parquet', index=True, engine='pyarrow')

### Write raw measurements per home to parquet files

In [ ]:
%%time 
for home_id in tqdm(list(df_meas.index.unique(level='id'))):
    df_meas.xs(home_id, drop_level=False).to_parquet(f'{home_id}_raw_measurements.parquet', index=True, engine='pyarrow')

## Put properties in separate columns, apply types and write parquet file(s)

In [ ]:
# unstacking takes the entire Twomes dataset uses 32 GB memory, so we have to do it home by home
del df_meas
gc.collect()

### Writing raw properties per home to a parquet file

In [ ]:
%%time
df_prop = pd.DataFrame()

for home_id in tqdm(homes_all):
    df_prop_home = Measurements.to_properties(
        pd.read_parquet(f'{home_id}_raw_measurements.parquet', engine='pyarrow'),
        twomes_full_properties_types
    )
    df_prop_home.to_parquet(f'{home_id}_raw_properties.parquet', index=True, engine='pyarrow')
    df_prop = pd.concat([df_prop, df_prop_home])   

In [ ]:
df_prop.info()

In [ ]:
df_prop

### Writing raw properties to a parquet file

In [ ]:
%%time 
df_prop.to_parquet('twomes_raw_properties.parquet', index=True, engine='pyarrow')